## Algoritmo de Edmonds-Karp


In [1]:
#Importaciones necesarias
from collections import defaultdict

In [2]:

# Se crea la clase Grafo con sus atributos y métodos
class Grafo:
    """
    Clase que representa un grafo dirigido con una capacidad en cada arco
    
    Atributos:
    - vertices: número de vértices del grafo
    - vecinos: lista de listas para almacenar los vecinos de cada vértice
    - capacidad: un diccionario anidado para almacenar las capacidades de los arcos 
                capacidad[u][v] indica la capacidad del arco de u a v
    """
    def __init__(self, vertices):
        """
        Constructor de la clase Grafo.
        Parámetros:
            - vertices: número de vértices del grafo
        """
        self.vertices = vertices
        self.vecinos = [[] for _ in range(vertices)]    
        self.capacidad = defaultdict(lambda: defaultdict(int))   

    def agregar_arco(self, u, v, capacidad):
        """
        Crea un arco del vértice u a v con la capacidad dada, si no estan en la lista de 
        vecinos se agregan.
        Parámetros:
            - u: vértice de origen
            - v: vértice de destino
            - capacidad: capacidad del arco
        """
        if v not in self.vecinos[u]:
            self.vecinos[u].append(v)
        # Arista inversa para recorrer el grafo para la red residual
        if u not in self.vecinos[v]:
            self.vecinos[v].append(u) 
        self.capacidad[u][v] = capacidad

    def obtener_vecinos(self, u):
        """
        Devuelve la lista de vecinos del vértice u.
        Parámetros:
            - u: vértice del cual queremos saber sus vecinos
        Retorna:
            - una lista con los vecinos de u.
        """
        return self.vecinos[u]
    
        

In [3]:


class EdmondsKarp:
    """
    Clase que implmenta el algoritmo Edmonds-Karp para encontrar el flujo máximo en un grafo dirigido. También 
    se crean las funciones necesarias para el algoritmo.
    Atributos:
        - grafo: un grafo dirigido de la clase Grafo.
        - vertices: número de vértices del grafo.
        - capacidad_original: almacena las capacidades iniciales de cada arco
    """
    def __init__(self, grafo: Grafo):
        """
        Constructor de la clase EdmondsKarp.
        Parámetros:
            - grafo: un grafo dirigido de la clase Grafo.
        """
        self.grafo = grafo
        self.vertices = grafo.vertices
        self.flujo = defaultdict(lambda: defaultdict(int))
        
        
        
    def capacidades_originales(self):    
        self.capacidad_original = {}
        for u in range(self.vertices):
            for v in self.grafo.obtener_vecinos(u):
                self.capacidad_original[(u, v)] = self.grafo.capacidad[u][v]


    def bfs(self, fuente, sumidero):
        """
        Función para realizar la búsqueda en anchura en un grafo dirigido.
        Parámetros:
            - fuente: vértice de origen del grafo y de la búsqueda.
            - sumidero: vértice de destino del grafo y de la búsqueda.
        Retorna:
            - padre: una lista con los vértices padres de cada vértice del camino. padre[u] = v 
            indica que v es el padre de u en el camino desde la fuente al sumidero. Si no se encuentra un
            camino, se retorna None.
        """
        padre = [None] * self.vertices
        visitado = [False] * self.vertices

        cola = [fuente]
        visitado[fuente] = True

        while cola:
            u = cola.pop(0)
            for v in self.grafo.obtener_vecinos(u):
                if not visitado[v] and self.grafo.capacidad[u][v] > 0:
                    visitado [v] = True
                    padre[v] = u
                    cola.append(v)

                    #Condición de fin
                    if v == sumidero:
                        return padre
           
        return None
    

    def edmonds_karp(self, fuente, sumidero):
        """
        Función que implementa el algoritmo de Edmonds-Karp para obtener el flujo maximo en un grafo
        dirigido.
        Parámetros:
            - fuente: vértice de origen del grafo.
            - sumidero : vértice de fin del grafo.
        Retorna: 
            - flujo: valor del flujo máximo.
        """
        #Guarda las capacidades originales antes de modificarse
        self.capacidades_originales()
        flujo = 0
        
        while True:
            padre = self.bfs(fuente, sumidero)
            if padre is None:
                break
            
            #padre tiene los padres de cada vértice por tanto vamos hacia atrás
            capacidad_minima = float('inf')
            v = sumidero
            #Se obtiene la capacidad mínima del camino
            while v != fuente:
                u = padre[v]
                capacidad_minima = min(capacidad_minima, self.grafo.capacidad[u][v])
                v = u
            
            #Se cambian las capacidades en los arcos del camino
            v = sumidero
            while v != fuente:
                u = padre[v]
                self.grafo.capacidad[u][v] -= capacidad_minima
                self.grafo.capacidad[v][u] += capacidad_minima
                self.flujo[u][v] += capacidad_minima
                self.flujo[v][u] -= capacidad_minima
                v = u

            flujo += capacidad_minima
        return flujo


    def obtener_caminos(self, fuente, sumidero):

        caminos = []
        #Copia del flujo para no modificar el original
        import copy
        flujo_restante = copy.deepcopy(self.flujo)

        while True:
            padre = [None] * self.vertices
            visitado = [False] * self.vertices

            cola = [fuente]
            visitado[fuente] = True
            encontrado = False
            
            # BFS para encontrar un camino con flujo disponible
            while cola and not encontrado:
                u = cola.pop(0)
                
                for v in self.grafo.obtener_vecinos(u):
                    if not visitado[v] and flujo_restante[u][v] > 0:
                        visitado[v] = True
                        padre[v] = u
                        cola.append(v)

                        # Si llegamos al sumidero, terminamos
                        if v == sumidero:
                            encontrado = True
                            break

            if not encontrado:
                break

            #Encontramos el flujo del camino
            camino = []
            v = sumidero
            flujo_min = float('inf')
            while v != fuente:
                u = padre[v]
                camino.append(v)
                flujo_min = min(flujo_min, flujo_restante[u][v])
                v = u

            camino.append(fuente)
            camino.reverse()

            #Restamos el flujo del camino al flujo total del arco
            for i in range(len(camino)-1):
                u, v = camino[i], camino[i+1]
                flujo_restante[u][v] -= flujo_min
                

            caminos.append((camino, flujo_min))
            
        return caminos
        
    


   


   

In [ ]:

g = Grafo(14)
g.agregar_arco(0, 4, 1)
g.agregar_arco(0, 2, 1)
g.agregar_arco(0, 8, 1)
g.agregar_arco(0, 9, 1)
g.agregar_arco(3, 13, 1)
g.agregar_arco(5, 13, 1)
g.agregar_arco(7, 13, 1)
g.agregar_arco(10, 13, 1)
g.agregar_arco(2, 3, 1)
g.agregar_arco(2, 5, 1)
g.agregar_arco(2, 7, 1)
g.agregar_arco(2, 10, 1)
g.agregar_arco(4, 5, 1)
g.agregar_arco(4, 10, 1)
g.agregar_arco(9, 5, 1)
g.agregar_arco(9, 7, 1)
g.agregar_arco(9, 10, 1)
g.agregar_arco(8, 10, 1)

##diccionario con las correspondencias de los vértices a los nombres de los paises


fuente = 0
sumidero = 13
flujo = EdmondsKarp(g)
max_flujo = flujo.edmonds_karp(fuente, sumidero)
print("El flujo máximo es:", max_flujo)
print("Caminos encontrados:", flujo.obtener_caminos(fuente, sumidero))


El flujo máximo es: 4
Caminos encontrados: [([0, 4, 5, 13], 1), ([0, 2, 3, 13], 1), ([0, 8, 10, 13], 1), ([0, 9, 7, 13], 1)]
